In [4]:
# Imports
import qiskit


In [41]:
# Set up the circuit
path = "test_circuits/3_17_13.qasm"

qc = qiskit.QuantumCircuit.from_qasm_file(path)
#qc.draw()
print(qc.data[1][1])
print(len(qc.data[1][1]))
print(qc.data[1][1][1].index)


[Qubit(QuantumRegister(16, 'q'), 0), Qubit(QuantumRegister(16, 'q'), 2)]
2
2


<ipython-input-41-29ef09a3f1c9>:8: DeprecationWarning: The property ``qiskit.circuit.bit.Bit.index`` is deprecated as of qiskit-terra 0.17. It will be removed no earlier than 3 months after the release date. Instead, use :meth:`~qiskit.circuit.quantumcircuit.QuantumCircuit.find_bit` to find all the containing registers within a circuit and the index of the bit within the circuit.
  print(qc.data[1][1][1].index)


In [ ]:
# Go through the quantum circuit and make a list of all the qubits that need to be entangled, in order

print(qc.data[1][1])
print(len(qc.data[1][1]))
print(qc.data[1][1][1].index)